# Assignment 02

TODO:
* Fragen ob bei suche nach mehreren Wörtern einfach summe gebildet werden kann?
* Daten der Suche abspeichern
* Doku
* Werden bei Werte bei PageRank und TFIDF Suche addiert oder mulitpliziert

In [55]:
import json
import os
import nltk
import string
import re
import pandas as pd
from IPython.display import display
import numpy as np
import math

## a) Implement a crawler

Der Crawler wurde mit hielfe der Bibliothek scrapy geschrieben. Der eigen implementierte Code ist in der Datei ~/sitespider/sitespider/spiders/site_spider.py zu finden und sieht wie folgt aus:

```python
# -*- coding: utf-8 -*-
import scrapy
import re
import json


class SiteSpiderSpider(scrapy.Spider):
    name = 'site_spider'

    start_urls = [
        'http://people.f4.htw-berlin.de/~zhangg/pages/teaching/pages/d01.html',
        'http://people.f4.htw-berlin.de/~zhangg/pages/teaching/pages/d06.html',
        'http://people.f4.htw-berlin.de/~zhangg/pages/teaching/pages/d08.html'
    ]

    def parse(self, response):
        id = response.url.split('/')[-1].split('.')[0]

        with open("sites/%s.json" % id, 'w') as f:
            data = {'id': id,
                    'text': "".join(line for line in response.css('body::text').extract()),
                    'url': response.url,
                    'back_links': [link.split('.')[0] for link in response.css('a::attr(href)').extract()]
                    }

            json.dump(data, f)

        for next_page in response.css('a::attr(href)').extract():
            yield response.follow(next_page, callback=self.parse)
```

Die Klasse SiteSpiderSpider erbt von der Klasse scrapy.Spider um später, die geschriebene Klasse mittels der scrapy Bibliothek als Crawler einsetzen zu könnnen.

Das Attribut "name" identifizieren den Crwaler und wird dazu benutzt, den Crawler über die Konsole aufzurufen. Das Attribut "start_urls" legt fest, welche URL's vom Crawler initial angefragt werden sollen. Die Methode parse() nimmt dann anschlißend die Rückgabe jeder dieser Anfrage entgegen und behandelt diese. Da die Methode parse() die default callback methode von scrapy ist, muss dies nicht explezit angegeben werden.
 
Die "parse()"-Methode erzeugt ein String, der die Anfrage eindeutig beschriebt. Dieser Wert wir in der Variablen "id" gespeichert und anschließend als Dateiname verwendet, in dem die Daten im JSON-Format der Anfrage gespeichert werden. In der Datei selber wird die Id, der eigentliche Text, der angefragten Seite ohne HTML-Tags, die URL, und alle auf der Seite gefundenen Links als Array, gespeichert. Anschließend wird für alle auf der Seite gefundenen Links der selbe Prozess mittels der Methode "response.follow(..)" angestoßen.

## b) Crawl all reachable documents starting from the following URLs

Der implementierte Crawler wird mittels des Konsolen-Befehls 'scrapy crawl site_spider' in dem Verzeichniss ~/sitespider aufgerufen. Die resultierenden Dateien sind in dem Unterordner ~/sitespider/sites/\* zufinden

### Importing crawled documents

Im folgenden Code-Block werden die Dateien mit den Daten der gecrawlten Seiten Datei für Datei ausgelesen und in der Variablen pages gespechert. Die Variable pages ist ein DictionaryArray, wobei jedes Dictionaty eine Seite representiert.

In [56]:
directory = 'sitespider/sites'
files = [x[2] for x in os.walk(directory)][0]
pages = []

for file in files:
    with open("%s/%s" % (directory, file)) as json_data:
        pages += [json.load(json_data)]

## c) Calculate the PageRanks of the downloaded pages

In diesem Abschnitt werden die PageRanks der Heruntergeladenen Seiten berechnet.

### Konstanten

Die folgenden Konstenten beschereiben folgendes:
* n = Anzahl der betrachteten Seiten
* t = Teleportationsrate
* d = Dämpfungsfaktor
* δ = Abbruchswert

In [57]:
n = len(pages)
t = 0.05
d =  1 - t
δ = 0.04 

### Functions

**PageRank**

Die Funktion calculate_page_rank() berechnet den PageRank für die übergebene Seite page_i. Folgende Formel wurde dabei implementiert:

$$ r_{k+1}(p_i) = d * \bigg ( \displaystyle\sum_{p_j \in B_{pi}} \frac{r_k(p_j)}{|p_j|} + \displaystyle\sum_{p_j,|p_j|=0} \frac{r_k(p_j)}{N} \bigg ) + \frac{t}{N} $$

In [58]:
def calculate_page_rank(page_i):
    sum_result = 0  
    for page_j in pages:
        if page_i['id'] in page_j['back_links']:
            sum_result += page_j['rank'] / len(page_j['back_links'])
        elif len(page_j['back_links']) == 0:
            sum_result += page_j['rank'] / n
    return d * sum_result + t / n

**Helper Functions**

In [59]:
def initialize():
    for page in pages:
        page['rank'] = 1/n

Die Funktion termination_condition() gibt einen boolischen Wert zurück, der sagt, ob die Abbruchsbedingung erfüllt ist oder nicht. Im Falle, dass es noch keine zwei Vergleichswerte gibt, also noch kein 'rank+1'-Schlüssel existiert, wird False zurückgegebn, ansonsten wird der Rückgabewert mit folgender Formel berechnet:

$$ \sum_{p_i,i \in 1,...,N} abs(r_{k+1}(p_i) - r_k(p_i)) <= \delta $$

In [60]:
def termination_condition():
    delta = 0
    for page in pages:
        if not 'rank+1' in page:
            return False
        else:
            delta += abs(page['rank+1'] - page['rank'])
    return  delta <= δ

Die Funktion set_rank() setzt für die übergebene Seite, falls der Schlüssel rank+1 existiert, den älteren PageRank-Wert auf den aktuellen Wert der Variable rank+1.

In [61]:
def set_rank(page):
    if not 'rank+1' in page:
        return
    else:
        page['rank'] = page['rank+1']

### Calculating PageRanks for all Documents

Im folgenden wird zunächst die Methode initialize() aufgerufen um alle Seiten mit einem initalen PageRank-Wert zu initialisieren. Anschließend wird so lange eine While-Schleife durchlaufen, bis die Abbruchsbedingung erfüllt ist. Innerhalb der While-Schleife wird jede Seite durchlaufen, wobei jeweils die Methode set_rank() aufgerufen wird und anschließend der nächste PageRank für dei Seite berechnet wird.

In [62]:
initialize()
while not termination_condition():
    for page in pages:
        set_rank(page)
        page['rank+1'] = calculate_page_rank(page)
    

### Saving PageRanks

Die Resultate werden in der Datei rank.txt gespeichert. Dabei steht zunächst die id der Seite und anscließedn der eigentliche Wert.

In [63]:
with open('rank.txt', 'w') as f:
    for page in pages:
        f.write("%s: %s\n" % (page['id'], page['rank+1']))

### Checking the sum of all PageRanks

Hier wird zusätzlich die Summe aller PageRanks berechnet, um zu prüfen, ob sich der Wert der Zahl 1 annähert.

In [64]:
rank_sum = 0
for page in pages:
    rank_sum += page['rank+1']
print(rank_sum)

1.021723309931979


## d) Build a tf-Index for the words contained in the documents

$$ w_{t,d} =
  \begin{cases}
    1 + log_{10}(tf_{t,d})       & \quad \text{if } tf_{t,d} > 0\\
    0 & \quad \text{otherwise}\\
  \end{cases}
$$

In [65]:
tf_dict = {}
term_set = set()
stopwords = []
exclude = set(string.punctuation)
porter = nltk.PorterStemmer()

with open('stop_words.txt') as line:
    stopwords += re.sub('[^a-zA-Z0-9,]', '', line.read()).split(',')

### Calculation of unique terms in documents

In [66]:
for page in pages:
    for term in nltk.word_tokenize(page['text']):
        if term not in exclude and term not in stopwords:
            term_set.add(porter.stem(term).lower())

### Calculation of Term Frequency

In [67]:
for page in pages:
    tf_dict[page['id']] = {}
    for term in term_set:
        tf_dict[page['id']][term] = 0
    for term in nltk.word_tokenize(page['text']):
        if term not in exclude and term not in stopwords:
            tf_dict[page['id']][porter.stem(term).lower()] += 1

### Functions

**Weighted Term Frequency**

In [68]:
def get_weighted_tf(doc_id, term):
    if term in tf_dict[doc_id]:
        if tf_dict[doc_id][term] == 0:
            return 0
        else:
            return 1 + math.log10(tf_dict[doc_id][term])
    else:
        return 0

### Saving Term Frequency (TF-Index)

In [69]:
tf_df = pd.DataFrame(tf_dict)
tf_df.to_csv('index.txt', header=True, index=True, sep=';')

## e) Implement a function search to search for documents containing given words

$$ idf_t = log_{10}( \frac{N}{df_t}) $$

$$ tfidf = tf * idf $$

In [70]:
df_dict = {}

### Calculation of Document Frequency

In [71]:
for page in pages:
    for term in nltk.word_tokenize(page['text']):
        if term not in exclude and term not in stopwords:
            if porter.stem(term).lower() in df_dict and page['id'] not in df_dict[porter.stem(term).lower()]['documents']:
                df_dict[porter.stem(term).lower()]['count'] += 1
                df_dict[porter.stem(term).lower()]['documents'] += [page['id']]
            elif porter.stem(term).lower() not in df_dict:
                df_dict[porter.stem(term).lower()] = {'count': 1,
                                                      'documents': [page['id']]
                                                     }          

### Functions

**Weighted Inverse Document Frequency**

In [72]:
def get_weighted_idf(term):
    if term in df_dict:
        if df_dict[term]['count'] == 0:
            return 0
        else:
            return math.log10( n / df_dict[term]['count'])
    else:
        return 0

**Weighted TF-IDF**

In [73]:
def get_weighted_tf_idf(doc_id, term):
    return get_weighted_tf(doc_id, term) * get_weighted_idf(term)

**Search**

In [74]:
def search(terms, page_rank=False):
    result = {}
    for page in pages:
        result[page['id']] = 0
        for term in terms:
            result[page['id']] += get_weighted_tf_idf(page['id'], porter.stem(term).lower())
            if page_rank:
                result[page['id']] *= page['rank+1']
    return result

### Searching with TF-IDF 

In [78]:
search_terms = [['token'],['index'],['classification'],['classification', 'token']]

with open('tfidf_search.txt', 'w') as f:
    for search_term in search_terms:
        f.write('Suchwort: %s\n\n' % ', '.join(search_term))
        result = search(search_term)
        for key in result:
            f.write("%s: %s\n" % (key, result[key]))
        f.write('\n\n')

## f) Extend your search function and include PageRank to score the documents

### Searching with TF-IDF and PageRank

In [79]:
search_terms = [['token'],['index'],['classification'],['classification', 'token']]

with open('pageranke_search.txt', 'w') as f:
    for search_term in search_terms:
        f.write('Suchwort: %s\n\n' % ', '.join(search_term))
        result = search(search_term, page_rank=True)
        for key in result:
            f.write("%s: %s\n" % (key, result[key]))
        f.write('\n\n')